**School Donation Analysis** 🏫

The below notebook is an attempt to use various tools available in **python** to work on the dataset

**School_Donations** which contains over*4687884* record of the **schools**

their various departments, the **teachers** involved in the donation drives

and the *states* of **United States** which organized projects in the school

for the purpose of **donation generation**


**Importing the Libraries**

**numpy** for working on arrays

**pandas** for working and creating our own datasets

**matplotlib, pyplot, cufflinks** for visualisation of results

**datetime** Datetime module supplies classes to work with date and time. These classes provide a number of functions to deal with dates, times and time intervals.

*An extensive use of **plotly and cufflinks** has been done*

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as of
import cufflinks as cf
import datetime as dt
%matplotlib inline

In [2]:
of.init_notebook_mode(connected = True)
cf.go_offline()

**Loading the Dataset**

In [3]:
donations = pd.read_csv('../input/school-donations/Donations.csv')
donors = pd.read_csv('../input/school-donations/Donors.csv')
projects = pd.read_csv('../input/school-donations/Projects.csv')
resources = pd.read_csv('../input/school-donations/Resources.csv')
schools = pd.read_csv('../input/school-donations/Schools.csv')
teachers = pd.read_csv('../input/school-donations/Teachers.csv')

**Describing the Dataset**

In [4]:
print('Shape of donations dataframe is:' , donations.shape)
print('Shape of donors dataframe is:' , donors.shape)
print('Shape of projects dataframe is:' , projects.shape)
print('Shape of resources dataframe is:' , resources.shape)
print('Shape of schools dataframe is:' , schools.shape)
print('Shape of teachers dataframe is:' , teachers.shape)

In [5]:
donations.head()

In [6]:
donors.head()

In [7]:
projects.head()

In [8]:
resources.head()

In [9]:
schools.head()

In [10]:
teachers.head()

In [11]:
donations.describe()

In [12]:
donors.describe()

In [13]:
projects.describe()

In [14]:
resources.describe()

In [15]:
schools.describe()

In [16]:
teachers.describe()

**Creating New Data**

In [17]:
data = pd.merge(donations , projects , how='inner' , on = 'Project ID')

In [18]:
data2 = pd.merge(data , donors , how='inner' , on='Donor ID')

In [19]:
data3 = pd.merge(data2 , schools , how='inner' , on='School ID')

In [20]:
data4 = pd.merge(data3, teachers , how='inner' , on='Teacher ID')

In [21]:
data4.head()

In [22]:
a = data4.columns.values.tolist()
a

The states that have most number of schools that opened project to gather donations:

In [23]:
s = schools['School State'].value_counts().sort_values(ascending = False).head(10)
s

In [24]:
s.iplot(kind='bar' , xTitle='States' , yTitle='Number of schools' , title='Number of schools involved in projects by states')

In [25]:
s2 = data4.groupby('School State')['Donation Amount'].mean().sort_values(ascending=False).head(10)
s2

In [26]:
s2.iplot(kind='bar' , xTitle='State' , yTitle='Average donation per project' 
         , title='Top 10 states(with maximum doantion)' , colorscale='paired' )

**Analysis of Data Using mean median and percentile functionalities**

In [27]:
mean = np.mean(data4['Donation Amount'].dropna())
median = np.median(data4['Donation Amount'].dropna())
percentiles = np.percentile(data4['Donation Amount'].dropna() ,[25,75])
minimum = data4['Donation Amount'].dropna().min()
maximum = data4['Donation Amount'].dropna().max()

print('mean donation amount is:' ,np.round(mean,2))
print('median donation amount is:' ,median)
print('25% and 75% donation amount is:' ,percentiles)
print('minimum donation amount is:' ,minimum)
print('maximum donation amount is:' ,maximum)

*It is quite clear from above statistics that our Donations Amount column have lots of outliers since mean is 60 whereas median is 25 which shows that there are plenty of outliers causing mean to rise, second indicator is that we have 25th and 75th percentiles both below than mean. In other words although %75 percent of our data smaller than 50 we have a mean values which is 60.66 which is also a good indicator of outliers. Lastly we can easily say that maximum value is a huge outlier too.*

In [28]:
x = np.sort(data4["Donation Amount"].dropna())
y = np.arange(1,len(x)+1)/len(x)
plt.plot(x,y,marker = '.')

In [29]:
s3 = data4.groupby('Donor State')['Donation ID'].count().sort_values(ascending = False).head(15)
s3

In [30]:
s3.iplot(kind='bar' , xTitle='State',yTitle='Number of donations',title='Donations count',colorscale='paired')

**More advanced analysis of the states and their Donations:**

In [31]:
s4 = schools['School State'].value_counts()
s5 = data4.groupby('Donor State')['Donation ID'].count()
df = pd.concat([s4,s5],axis=1,keys=['Projects','Donations'])

In [32]:
df = df.dropna()

In [33]:
df.head()

In [34]:
df.iplot(kind='scatter',xTitle='Projects',
         yTitle='Donations',title='Projects vs Donations',
         symbol='x',colorscale='paired',mode='markers')

***A linear model is used to show the relationship between projects and donations.***

In [35]:
slope,intercept = np.polyfit(df.Projects,df.Donations,1)
x = np.array([df.Projects.min(),df.Projects.max()])
y = slope*x + intercept
plt.plot(x,y)

**Combining the plots**

In [36]:
df.plot.scatter(x='Projects' , y='Donations')
slope,intercept = np.polyfit(df.Projects,df.Donations,1)
x = np.array([df.Projects.min(),df.Projects.max()])
y = slope*x + intercept
plt.plot(x,y)
plt.tight_layout()
plt.margins(0.05)

In [37]:
data4.head(2)

In [38]:
s6 = data4["Project Type"].value_counts()
s6

In [39]:
s7 = data4.groupby('Project Type')['Donation Amount'].sum().astype(int)
s7

In [40]:
plt.subplot(2,1,1)
plt.pie(s6 , startangle=90)
plt.subplot(2,1,2)
plt.pie(s7 , startangle=90)
plt.tight_layout()
plt.margins(0.05)
fig = plt.gcf()
fig.set_size_inches(25,15)

In [41]:
data4['Project Subject Category Tree'].nunique()

In [42]:
s8 = data4.groupby('Project Subject Category Tree')['Donation Amount'].sum().astype(int).sort_values(ascending = False).head(15)
s8

In [43]:
s9 = s8/1000000
s9.iplot(kind="bar" , xTitle='Project sub category' , yTitle='Donation amount in millions',
        title='Donation amount by project subject' , colorscale='paired')

In [44]:
data4[['Project Posted Date' , 'Project Fully Funded Date']].isnull().sum()

In [45]:
data4[['Project Posted Date' , 'Project Fully Funded Date']].head()

In [46]:
data4['Project Posted Date'] = pd.to_datetime(data4['Project Posted Date'])

In [47]:
data4['Project Fully Funded Date'] = pd.to_datetime(data4['Project Fully Funded Date'])

In [48]:
data4['Funding Time'] = data4['Project Fully Funded Date'] - data4['Project Posted Date'] 
data4[['Funding Time','Project Posted Date' , 'Project Fully Funded Date']].head()

In [49]:
data4[['Funding Time','Project Posted Date' , 'Project Fully Funded Date']].isnull().sum()

In [50]:
data5 = data4[pd.notnull(data4['Funding Time'])]
data5[['Funding Time','Project Posted Date' , 'Project Fully Funded Date']].isnull().sum()

In [51]:
data5['Funding Time'] = data5['Funding Time'].dt.days

In [52]:
data5[['Funding Time','Project Posted Date' , 'Project Fully Funded Date']].head()

In [53]:
wrong_overall_mean_time = data5['Funding Time'].mean()
wrong_overall_mean_time

In [54]:
overall_mean_time = data5.groupby('Project ID')['Funding Time'].mean()
output = overall_mean_time.mean()
output

In [55]:
#Average funding time for each state

state_project_funding_time = data5.groupby(['School State' , 'Project ID'])['Funding Time'].mean()
state_project_funding_time

In [56]:
state_average_project_funding_time = state_project_funding_time.groupby('School State').mean()
state_average_project_funding_time.round(0)

In [57]:
fast = state_average_project_funding_time.round(0)
fast[fast<32].sort_values().head(10)

In [58]:
fast_funding = fast[fast<32].sort_values().head(10)
fast_funding.iplot(kind='bar' , xTitle='States' , yTitle='fully funding time(in days)',
                  title='states that fund projects earlier than others',
                  colorscale='paired')

In [59]:
slow = state_average_project_funding_time.round(0)
slow[slow>32].sort_values(ascending = False).head(10)

In [60]:
slow_funding = slow[slow>32].sort_values(ascending = False).head(10)
slow_funding.iplot(kind='bar' , xTitle='States' , yTitle='fully funding time(in days)',
                  title='states that fund projects earlier than others'
                  )